# Productive and Reliable Constitutive Modeling in Rust

## **Jed Brown**, CU Boulder
### Collaborators: Manuel Drehwald, Leila Ghaffari, Tobin Isaac, William Moses


## Scientific Computing in Rust 2024-07-19

In [3]:
from IPython.display import SVG, Video, HTML, IFrame
import pandas as pd
import altair as alt
from io import StringIO
import numpy as np


# Nonlinear computational mechanics

<video src="figures/ratel/tire-platen.webm" autoplay loop />
<video src="figures/ratel/schwarz-q2-5x5x5-t20-l2-r2.webm" width="85%" autoplay controls loop />

## "Write once" infrastructure

* Algebraic solvers, finite element primitives
* Communication, IO
## High-wear surfaces
* Materials, equations of state
* Parametrizations of subgrid processes
* Boundary conditions

<img src="figures/Boeing_A2_isoQspeed2_lowRes.png" width="100%" />

# How are constitutive models used?

<img src="figures/crikit-workflow.svg" width="100%" />

* Every nonlinear iteration or time step of a simulation
  * Every quadrature point in every element of a simulation (millions or billions of points)
* Must run on a GPU if the solver is using GPUs

# What properties must constitutive models have?

* Dimensional consistency/invariance
* Reference-frame invariance/equivariance
\begin{align}
\text{invariant} && \psi(\mathbf E) &= \psi(Q \mathbf E Q^T) & \forall Q \in O(3) \\
\text{equivariant} && Q \mathbf S(\mathbf E) Q^T &= \mathbf S(Q \mathbf E Q^T) & \forall Q \in O(3)
\end{align}

## Integrity bases can represent all equivariant functions
<img src="figures/wineman-pipkin.svg" width="100%" />

# What is the state of practice?

* Materials scientists like to model in terms of:
  * Free energy
  * Dissipation potential
* Derivatives are required for observable relations
* Further derivatives for efficient solvers
* Many publications per year:
  * dedicate lots of space to representation of derivatives
  * software interfaces are error-prone

* Heavy C++ hierarchies, hard to reuse
* Tedious code in Fortran (e.g., for Abaqus `UMAT`/`UHYPER`)
<img src="figures/ratel/abaqus-uhyper.png" />

# Enzyme `#[feature(autodiff)]` for Neo-Hookean model

$$\DeclareMathOperator{\trace}{trace}
\psi(\mathbf E) = \frac{\lambda}{4}(J^2 - 1 - 2\ln J) + \mu (\trace \mathbf E - \ln J)$$
where $\mathbf E$ is Green-Lagrange strain and $J = \sqrt{\lvert \mathbf I + 2\mathbf E \rvert}$.
```rs
#[autodiff(d_psi, Reverse, Duplicated, Const, Active)]
fn psi(e: &KelvinMandel, nh: &NeoHookean) -> f64 {
    let J = e.cauchy_green().det().sqrt();
    let lnJ = J.ln();
    0.25 * nh.lambda * (J * J - 1. - 2. * lnJ) + nh.mu * (e.trace() - lnJ)
}
```

Compute stress $\mathbf \tau(\mathbf e) = \frac{\partial \psi}{\partial \mathbf e} \mathbf b$ for current configuration, with its derivative for use by Newton solvers:
```rs
#[autodiff(d_stress, Forward, Dual, Const, Dual)]
fn stress(e: &KelvinMandel, nh: &NeoHookean, tau: &mut KelvinMandel) {
    let mut dpsi_de = KelvinMandel::zero();
    d_psi(&e, &mut dpsi_de, &nh, 1.0);
    let b = e.cauchy_green();
    *tau = dpsi_de * b;
}
```

# Diman: Zero-cost compile-time dimensional analysis (Toni Peter)

```rs
struct Primitive {
    pressure: Pressure<f64>,
    velocity: [Velocity<f64>; 3],
    temperature: Temperature<f64>,
}
struct Conservative {
    density: MassDensity<f64>,
    momentum: [MomentumDensity<f64>; 3],
    energy: EnergyDensity<f64>,
}
```

In gas dynamics, one needs to convert between primitive and conservative variables. This depends on the equation of state (gas model), which is an active research area (e.g., [CoolProp](http://coolprop.org/fluid_properties/PurePseudoPure.html#introduction)).

```rs
impl From<&Primitive> for Conservative {
    fn from(s: &Primitive) -> Self {
        let gas = IdealGas::air();
        let density: MassDensity<f64> = s.pressure / (gas.r * s.temperature);
        let energy_internal: SpecificEnergy<f64> = gas.cv * s.temperature;
        let energy_kinetic =
            0.5 * s.velocity.iter().map(|v| v * v).sum::<SpecificEnergy<f64>>();
        let momentum = s.velocity.map(|v| density * v);
        let energy = density * (energy_internal + energy_kinetic);
        Self {density, momentum, energy}
    }
}

#[autodiff(d_primitive_to_conservative, Forward, Dual, Dual)]
fn primitive_to_conservative(p: &Primitive, c: &mut Conservative) {
    *c = Conservative::from(p);
}
```

# Diman prevents coding bugs

<img src="figures/diman/diman-mismatch.png" />

## Diman works seamlessly with `#[feature(autodiff)]`

<img src="figures/diman/primitive_to_conservative.png" />

# A day in the life

## What I dream I do
### cross-cutting methods and community software

## What my students think I do
### review pull requests

## What my university thinks I do
### teach classes

## What I actually do

### debug broken environments, linker errors, and memory errors via email during faculty meetings

# What's next?

## Implicit materials
* Instead of $y = f(x)$, we have $f(y; x) = 0$ (or a complimentarity problem).
* Plasticity, some rheology
* Needs rootfinder/optimizer to run at each quadrature point.
* Often $6\times 6$ matrices in solve, bigger for multiscale models.
* Prior C++ art: [SNLS: Small Non-Linear Solver](https://github.com/LLNL/SNLS) (Robert Carson)
* Solver should be dimensionally-aware (even if shed for linear algebra)
* Need custom derivatives in `#[feature(autodiff)]`

## GPU support

* libCEED performs run-time kernel fusion between library and user parts.
* Prototyping mixed-source LTO using LLVM IR. Related to Rust's new bitcode linker.
* `no_std` is nice for testing on CPU before moving to GPU.
* Rootfinder/optimizer needs `no_std` support.

## Releasing

* How stand-alone for material libraries?
* Smooth out build with Ratel